#### Next two cells are only needed for a Google Colab environment.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd '/content/drive/My Drive/CodingProjects/skateboard_trick_classification'

/content/drive/My Drive/CodingProjects/skateboard_trick_classification


In [0]:
import numpy as np

from keras.callbacks import EarlyStopping
from keras.layers import concatenate, Dense, Dropout, GlobalAveragePooling3D, Softmax
from keras.models import load_model, Model
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

from utils import config
from utils.data_generator import DataGenerator
from utils.i3d_inception import Inception_Inflated3d

## Data Generators

In [0]:
training_generator = DataGenerator(config.VIDEO_TRAINING_DIR, 
                                   config.RGB_TRAINING_BATCH_SIZE, 
                                   is_training=True, rgb_data_only=True)

validation_generator = DataGenerator(config.VIDEO_VALIDATION_DIR, 
                                     config.RGB_VALIDATION_BATCH_SIZE, 
                                     is_training=False, rgb_data_only=True)

test_generator = DataGenerator(config.VIDEO_TEST_DIR, 
                               config.RGB_TEST_BATCH_SIZE, 
                               is_training=False, rgb_data_only=True)

# Train RGB Model

In [0]:
input_shape = (None, config.RGB_FRAME_HEIGHT, config.RGB_FRAME_WIDTH, config.CHANNELS)
i3d_model = Inception_Inflated3d(include_top=False, weights='rgb_imagenet_and_kinetics', 
                                 input_shape=input_shape, 
                                 classes=config.RGB_N_CLASSES)
x = Dropout(0.5)(i3d_model.output)
x = Dense(config.RGB_N_CLASSES, name='rgb_predictions')(x)
output = Softmax()(x)

model = Model(inputs=i3d_model.input, outputs=output)
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
early_stopping = EarlyStopping(patience=4, restore_best_weights=True, verbose=1)
model.fit_generator(training_generator, 
                    epochs=100,
                    validation_data=validation_generator, 
                    callbacks=[early_stopping])

In [0]:
# recompiling the model will reduce the size of the saved model by resetting the optimizer state
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.save(config.RGB_MODEL_FILEPATH)

# Evaluate Model

In [0]:
model = load_model(config.RGB_MODEL_FILEPATH)

## Validation Set

In [0]:
y_true = validation_generator.labels
predictions = model.predict_generator(validation_generator)
y_pred = np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred, 
                               target_names=config.RGB_CLASS_NAMES, 
                               digits=4)
print(report)
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

    kickflip     0.5714    0.5581    0.5647        43
360_kickflip     0.6154    0.5714    0.5926        42
       50-50     0.4800    0.5333    0.5053        45
   nosegrind     0.4043    0.4419    0.4222        43
  boardslide     0.6216    0.5476    0.5823        42
   tailslide     0.5833    0.6087    0.5957        46
        fail     0.8718    0.8293    0.8500        41

    accuracy                         0.5828       302
   macro avg     0.5925    0.5843    0.5875       302
weighted avg     0.5897    0.5828    0.5853       302

[[24 10  2  4  2  1  0]
 [12 24  0  5  0  0  1]
 [ 1  0 24  5  5  9  1]
 [ 3  0 10 19  4  4  3]
 [ 0  0  9  4 23  6  0]
 [ 1  0  5  9  3 28  0]
 [ 1  5  0  1  0  0 34]]


## Test Set

In [7]:
y_true = test_generator.labels
predictions = model.predict_generator(test_generator)
y_pred = np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred, 
                               target_names=config.RGB_CLASS_NAMES, 
                               digits=4)
print(report)
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

    kickflip     0.7143    0.4000    0.5128        25
360_kickflip     0.5909    0.5200    0.5532        25
       50-50     0.4500    0.3600    0.4000        25
   nosegrind     0.3611    0.5200    0.4262        25
  boardslide     0.3667    0.4400    0.4000        25
   tailslide     0.3750    0.4800    0.4211        25
        fail     1.0000    0.8400    0.9130        25

    accuracy                         0.5086       175
   macro avg     0.5511    0.5086    0.5180       175
weighted avg     0.5511    0.5086    0.5180       175

[[10  7  0  4  1  3  0]
 [ 4 13  1  3  2  2  0]
 [ 0  0  9  2  5  9  0]
 [ 0  1  4 13  6  1  0]
 [ 0  0  5  5 11  4  0]
 [ 0  0  1  9  3 12  0]
 [ 0  1  0  0  2  1 21]]
